In [2]:
from dotenv import load_dotenv
from pathlib import Path
load_dotenv()
print("✅ Setup complete")

✅ Setup complete


In [3]:
from langchain_core.vectorstores import InMemoryVectorStore
# from langchain_openai import OpenAIEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.documents import Document

# Initialize
# embeddings = GoogleGenerativeAIEmbeddings(model="text-embedding-3-small")
embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")
vector_store = InMemoryVectorStore(embedding=embeddings)

# Add documents
docs = [
    Document(page_content="Python is a programming language"),
    Document(page_content="Machine learning uses algorithms"),
    Document(page_content="The sun is a star")
]

vector_store.add_documents(docs)
print(f"✅ Added {len(docs)} documents\n")

# Search
results = vector_store.similarity_search("What is Python?", k=2)

print("Search Results:")
for i, doc in enumerate(results, 1):
    print(f"{i}. {doc.page_content}")

✅ Added 3 documents

Search Results:
1. Python is a programming language
2. Machine learning uses algorithms


In [4]:
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

# Load and split a PDF
pdf_path = "attention.pdf"

if Path(pdf_path).exists():
    print("Loading PDF...")
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    
    # Split
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    chunks = splitter.split_documents(documents)
    print(f"Split into {len(chunks)} chunks")
    
    # Create FAISS vectorstore
    print("Creating embeddings (this may take a minute)...")
    vectorstore = FAISS.from_documents(chunks, embeddings)
    
    # Save to disk
    vectorstore.save_local("./faiss_index_notebook")
    print("✅ FAISS index saved\n")
    
    # Search
    query = "What is attention mechanism?"
    results = vectorstore.similarity_search(query, k=3)
    
    print(f"Query: {query}\n")
    for i, doc in enumerate(results, 1):
        print(f"Result {i}:")
        print(f"  {doc.page_content[:150]}...\n")
else:
    print(f"❌ PDF not found: {pdf_path}")

Loading PDF...
Split into 52 chunks
Creating embeddings (this may take a minute)...
✅ FAISS index saved

Query: What is attention mechanism?

Result 1:
  3.2 Attention
An attention function can be described as mapping a query and a set of key-value pairs to an output,
where the query, keys, values, and ...

Result 2:
  Scaled Dot-Product Attention
 Multi-Head Attention
Figure 2: (left) Scaled Dot-Product Attention. (right) Multi-Head Attention consists of several
att...

Result 3:
  The Transformer uses multi-head attention in three different ways:
• In "encoder-decoder attention" layers, the queries come from the previous decoder...



In [5]:
# Load existing index
if Path("./faiss_index_notebook").exists():
    loaded_vectorstore = FAISS.load_local(
        "./faiss_index_notebook",
        embeddings,
        allow_dangerous_deserialization=True
    )
    print("✅ Loaded existing FAISS index")
    
    # Use it
    results = loaded_vectorstore.similarity_search("transformers", k=2)
    print(f"\nFound {len(results)} results")
else:
    print("No existing index found")

✅ Loaded existing FAISS index

Found 2 results


In [6]:
results

[Document(id='4c88f6e9-67e9-4fab-add0-144ff8ef2ba5', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-04-10T21:11:43+00:00', 'author': '', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'attention.pdf', 'total_pages': 15, 'page': 2, 'page_label': '3'}, page_content='Figure 1: The Transformer - model architecture.\nThe Transformer follows this overall architecture using stacked self-attention and point-wise, fully\nconnected layers for both the encoder and decoder, shown in the left and right halves of Figure 1,\nrespectively.\n3.1 Encoder and Decoder Stacks\nEncoder: The encoder is composed of a stack of N = 6 identical layers. Each layer has two\nsub-layers. The first is a multi-head self-attention mechanism, and the second is a simple, position-\nwise fully connec

In [7]:
from langchain_chroma import Chroma

# Create Chroma vectorstore
chroma_store = Chroma(
    collection_name="my_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_db_notebook"
)

# Add documents
sample_docs = [
    Document(
        page_content="RAG combines retrieval and generation",
        metadata={"topic": "rag", "difficulty": "intermediate"}
    ),
    Document(
        page_content="LangChain simplifies LLM applications",
        metadata={"topic": "langchain", "difficulty": "beginner"}
    )
]

chroma_store.add_documents(sample_docs)
print("✅ Added documents to Chroma\n")

# Search with metadata filter
results = chroma_store.similarity_search(
    "Tell me about RAG",
    k=2,
    filter={"topic": "rag"}
)

print("Filtered search results:")
for doc in results:
    print(f"  {doc.page_content}")
    print(f"  Metadata: {doc.metadata}")

✅ Added documents to Chroma

Filtered search results:
  RAG combines retrieval and generation
  Metadata: {'difficulty': 'intermediate', 'topic': 'rag'}
